# 1. 이미지 벡터 추출 함수 정의

In [1]:
#이미지 특징 벡터 함수  추출
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
#코드 출처 https://becominghuman.ai/extract-a-feature-vector-for-any-image-with-pytorch-9717561d1d4c
# https://daeun-computer-uneasy.tistory.com/85

# Load the pretrained model
model = models.resnet18(pretrained=True)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    img = img.convert('RGB')
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros([1, 512, 1, 512])
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

#cosine similarity 함수 정의
def cos_pytorch(A, B):
  cos= nn.CosineSimilarity(dim=1, eps=1e-6)
  cos_sim = cos(A, B)
  return cos_sim.numpy()[0][0][0]

c:\Users\minje\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\minje\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 2. 인플루언서 특징 및 태깅 전처리

In [2]:
import pandas as pd
influencer_tag = pd.read_csv('C:/Korea university/공모전/해커톤/모델링/업체-인플루언서/성별포함인플루언서ID_리스트_상세태깅추가.csv')
tag = influencer_tag.copy()
tag.head()

,id,img_name,상세이미지_구도1,상세이미지_구도2,카테고리,팔로워,성별,구도,태그
0,@__02x02,"['2912436941397721496.jpg', '29132490604646296...","[전,전,전,전,전,전,상,전,전,전,전,전,전,전,전,전,상,전,전,하,전,전,전...","[야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야...",NaN,micro,여,"코디,야외,전신,상품","스트릿,레트로"
1,@__my_t__,"['2967674458791156213.jpg', '29820661560695805...","[전,전,전,전,전,전,전,니,전,전,전,전,전,전,전,전,니,전,전,상,전,전,니...","[실,야,실,실,실,실,실,실,야,실,실,실,실,실,실,실,실,실,실,실,실,실,실...",NaN,micro,남,"일상,야외,상품,전신","스트릿,캐주얼,댄디"
2,@__v.yuum_look__,"['2875249160037008816.jpg', '29141202885707093...","[니,니,상,니,니,니,니,니,니,니,니,니,니,니,니,니,니,상,니,니,니,니,니...","[실,야,실,실,실,실,실,실,실,야,실,실,실,실,실,실,실,야,실,실,실,실,실...",NaN,micro,여,"거울,상반신,스튜디오,일상","캐주얼,로맨틱"
3,@_jongh0,"['2970996238523862884.jpg', '29717036134133031...","[전,전,전,전,전,전,전,니,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전...","[야,야,야,실,야,실,야,야,야,야,야,야,야,실,야,야,야,야,실,야,야,야,야...",NaN,midtier,남,"코디,전신","캐주얼,스트릿"
4,@_mgi_closet_,"['2856667809394379675.jpg', '28587777545668397...","[전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전...","[실,실,실,실,야,야,실,실,실,실,실,실,야,실,실,실,실,야,실,실,실,실,야...",NaN,nano,남,"전신,야외,코디","댄디,스트릿"


## 2-1. 구도 및 태그 종류 확인

In [3]:
#구도 카테고리들
t =[]
for i in tag['구도']:
    t += i.split(',')
set(t)

{'거울', '상반신', '상품', '스튜디오', '야외', '일상', '전신', '코디'}

In [4]:
#테그 카테고리들
t =[]
for i in tag['태그']:
    t += i.split(',')
set(t)

{'걸리시', '댄디', '레트로', '로맨틱', '스트릿', '스포츠', '시크', '아메리칸캐주얼', '캐주얼', '포멀'}

In [ ]:
# 성별: 남(0), 여(1)
# 팔로워: 'mega', 'macro', 'midtier', 'micro'
# 구도: '거울', '상반신', '상품', '스튜디오', '야외', '일상', '전신', '코디'
# 카테고리(남): '댄디', '스트릿', '스포츠', '아메리칸캐주얼', '캐주얼', '포멀'
# 카테고리(여): '걸리시', '레트로', '로맨틱', '스트릿', '스포츠', '시크', '캐주얼', '포멀'

## 2-2. 특징들 리스트로 저장

In [5]:
for i in range(len(tag)):
    tag['구도'][i] = tag['구도'][i].split(',')
    tag['태그'][i] = tag['태그'][i].split(',')
tag.head()

C:\Users\minje\AppData\Local\Temp\ipykernel_24068\1496508183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['구도'][i] = tag['구도'][i].split(',')
C:\Users\minje\AppData\Local\Temp\ipykernel_24068\1496508183.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['태그'][i] = tag['태그'][i].split(',')


,id,img_name,상세이미지_구도1,상세이미지_구도2,카테고리,팔로워,성별,구도,태그
0,@__02x02,"['2912436941397721496.jpg', '29132490604646296...","[전,전,전,전,전,전,상,전,전,전,전,전,전,전,전,전,상,전,전,하,전,전,전...","[야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야,야...",NaN,micro,여,"[코디, 야외, 전신, 상품]","[스트릿, 레트로]"
1,@__my_t__,"['2967674458791156213.jpg', '29820661560695805...","[전,전,전,전,전,전,전,니,전,전,전,전,전,전,전,전,니,전,전,상,전,전,니...","[실,야,실,실,실,실,실,실,야,실,실,실,실,실,실,실,실,실,실,실,실,실,실...",NaN,micro,남,"[일상, 야외, 상품, 전신]","[스트릿, 캐주얼, 댄디]"
2,@__v.yuum_look__,"['2875249160037008816.jpg', '29141202885707093...","[니,니,상,니,니,니,니,니,니,니,니,니,니,니,니,니,니,상,니,니,니,니,니...","[실,야,실,실,실,실,실,실,실,야,실,실,실,실,실,실,실,야,실,실,실,실,실...",NaN,micro,여,"[거울, 상반신, 스튜디오, 일상]","[캐주얼, 로맨틱]"
3,@_jongh0,"['2970996238523862884.jpg', '29717036134133031...","[전,전,전,전,전,전,전,니,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전...","[야,야,야,실,야,실,야,야,야,야,야,야,야,실,야,야,야,야,실,야,야,야,야...",NaN,midtier,남,"[코디, 전신]","[캐주얼, 스트릿]"
4,@_mgi_closet_,"['2856667809394379675.jpg', '28587777545668397...","[전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전,전...","[실,실,실,실,야,야,실,실,실,실,실,실,야,실,실,실,실,야,실,실,실,실,야...",NaN,nano,남,"[전신, 야외, 코디]","[댄디, 스트릿]"


In [6]:
#불러온 데이터 전처리 (리스트가 str으로 변해있음..)
import ast
for i in range(len(tag)):
    tag['img_name'][i] = ast.literal_eval(tag['img_name'][i])
    tag['상세이미지_구도1'][i] = tag['상세이미지_구도1'][i].split('[')[1].split(']')[0].split(',')
    tag['상세이미지_구도2'][i] = tag['상세이미지_구도2'][i].split('[')[1].split(']')[0].split(',')

C:\Users\minje\AppData\Local\Temp\ipykernel_24068\4282012428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['img_name'][i] = ast.literal_eval(tag['img_name'][i])
C:\Users\minje\AppData\Local\Temp\ipykernel_24068\4282012428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['상세이미지_구도1'][i] = tag['상세이미지_구도1'][i].split('[')[1].split(']')[0].split(',')
C:\Users\minje\AppData\Local\Temp\ipykernel_24068\4282012428.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [7]:
tag.head()

,id,img_name,상세이미지_구도1,상세이미지_구도2,카테고리,팔로워,성별,구도,태그
0,@__02x02,"[2912436941397721496.jpg, 2913249060464629697....","[전, 전, 전, 전, 전, 전, 상, 전, 전, 전, 전, 전, 전, 전, 전, ...","[야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, ...",NaN,micro,여,"[코디, 야외, 전신, 상품]","[스트릿, 레트로]"
1,@__my_t__,"[2967674458791156213.jpg, 2982066156069580582....","[전, 전, 전, 전, 전, 전, 전, 니, 전, 전, 전, 전, 전, 전, 전, ...","[실, 야, 실, 실, 실, 실, 실, 실, 야, 실, 실, 실, 실, 실, 실, ...",NaN,micro,남,"[일상, 야외, 상품, 전신]","[스트릿, 캐주얼, 댄디]"
2,@__v.yuum_look__,"[2875249160037008816.jpg, 2914120288570709385....","[니, 니, 상, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, ...","[실, 야, 실, 실, 실, 실, 실, 실, 실, 야, 실, 실, 실, 실, 실, ...",NaN,micro,여,"[거울, 상반신, 스튜디오, 일상]","[캐주얼, 로맨틱]"
3,@_jongh0,"[2970996238523862884.jpg, 2971703613413303190....","[전, 전, 전, 전, 전, 전, 전, 니, 전, 전, 전, 전, 전, 전, 전, ...","[야, 야, 야, 실, 야, 실, 야, 야, 야, 야, 야, 야, 야, 실, 야, ...",NaN,midtier,남,"[코디, 전신]","[캐주얼, 스트릿]"
4,@_mgi_closet_,"[2856667809394379675.jpg, 2858777754566839754....","[전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, ...","[실, 실, 실, 실, 야, 야, 실, 실, 실, 실, 실, 실, 야, 실, 실, ...",NaN,nano,남,"[전신, 야외, 코디]","[댄디, 스트릿]"


## 2-3. 상세이미지 특징 전처리

* 분류
1. 성별 = 남,여
2. 상세이미지_구도1 = ['니샷', '상반신', '셀카', '제품', '전신', '하반신'] 
3. 상세이미지_구도2 = ['실내', '야외']
4. 카테고리 (남, 여에 따라 아래 두가지)\
    cate_m(sex=0인 경우) = ['댄디', '스트릿', '(포)스포츠', '아메리칸캐주얼', '캐주얼', '포멀']\
    cate_w(sex=1인 경우) = ['걸리시', '레트로', '로맨틱', '스트릿', '(츠)스포츠', '시크', '캐주얼', '포멀']

각 계정별로 주로 어떤 구도의 이미지가 있는지 count 한다

In [8]:
from collections import Counter
Counter(tag['상세이미지_구도1'][0])

Counter({'전': 27, '상': 2, '하': 1})

In [9]:
from collections import Counter

counts=[]
for i in tag['상세이미지_구도1']:
    count = Counter(i)
    counts.append(count)
tag['촬영구도 카운트'] = counts

counts1=[]
for i in tag['상세이미지_구도2']:
    count = Counter(i)
    counts1.append(count)
tag['촬영장소 카운트'] = counts1

## 2-3. one-hot encoding하기

In [11]:
follower = ['mega', 'macro', 'midtier', 'micro']
structure1 = ['니', '상', '셀', '제', '전', '하'] #['니샷', '상반신', '셀카', '제품', '전신', '하반신']
structure2 = ['실', '야'] #['실내', '야외']
cate_m = ['댄디', '스트릿', '스포츠', '아메리칸캐주얼', '캐주얼', '포멀']
cate_w = ['걸리시', '레트로', '로맨틱', '스트릿', '스포츠', '시크', '캐주얼', '포멀']
cate_total = ['걸리시', '댄디', '레트로', '로맨틱', '스트릿', '스포츠', '시크', '아메리칸캐주얼', '캐주얼', '포멀']

In [12]:
#팔로워 수
for i in range(len(tag)):
    encode = []
    for j in follower:
        if j in tag['팔로워'][i]:
            encode.append(1)
        else:
            encode.append(0)
    tag['팔로워'][i] = encode

C:\Users\minje\AppData\Local\Temp\ipykernel_24068\3067388280.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['팔로워'][i] = encode


In [13]:
#성별
for i in range(len(tag)):
    if tag['성별'][i] == '남':
        tag['성별'][i] = 0
    else:
        tag['성별'][i] = 1

C:\Users\minje\AppData\Local\Temp\ipykernel_24068\4257718039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['성별'][i] = 1
C:\Users\minje\AppData\Local\Temp\ipykernel_24068\4257718039.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['성별'][i] = 0


### 촬영구도, 촬영장소 빈도수 만큼 one-hot encoding
ex) ['니샷', '상반신', '셀카', '제품', '전신', '하반신'] -> [5,24,0,1,1,0]

In [15]:
tag['촬영구도 원핫'] = [0]*len(tag)
tag['촬영장소 원핫'] = [0]*len(tag)

In [23]:
tag.head()

,id,img_name,상세이미지_구도1,상세이미지_구도2,카테고리,팔로워,성별,구도,태그,촬영구도 카운트,촬영장소 카운트,촬영구도 원핫,촬영장소 원핫
0,@__02x02,"[2912436941397721496.jpg, 2913249060464629697....","[전, 전, 전, 전, 전, 전, 상, 전, 전, 전, 전, 전, 전, 전, 전, ...","[야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, 야, ...",NaN,"[0, 0, 0, 1]",1,"[코디, 야외, 전신, 상품]","[스트릿, 레트로]","{'전': 27, '상': 2, '하': 1}",{'야': 30},0,0
1,@__my_t__,"[2967674458791156213.jpg, 2982066156069580582....","[전, 전, 전, 전, 전, 전, 전, 니, 전, 전, 전, 전, 전, 전, 전, ...","[실, 야, 실, 실, 실, 실, 실, 실, 야, 실, 실, 실, 실, 실, 실, ...",NaN,"[0, 0, 0, 1]",0,"[일상, 야외, 상품, 전신]","[스트릿, 캐주얼, 댄디]","{'전': 23, '니': 6, '상': 1}","{'실': 27, '야': 2}",0,0
2,@__v.yuum_look__,"[2875249160037008816.jpg, 2914120288570709385....","[니, 니, 상, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, 니, ...","[실, 야, 실, 실, 실, 실, 실, 실, 실, 야, 실, 실, 실, 실, 실, ...",NaN,"[0, 0, 0, 1]",1,"[거울, 상반신, 스튜디오, 일상]","[캐주얼, 로맨틱]","{'니': 26, '상': 3}","{'실': 26, '야': 3}",0,0
3,@_jongh0,"[2970996238523862884.jpg, 2971703613413303190....","[전, 전, 전, 전, 전, 전, 전, 니, 전, 전, 전, 전, 전, 전, 전, ...","[야, 야, 야, 실, 야, 실, 야, 야, 야, 야, 야, 야, 야, 실, 야, ...",NaN,"[0, 0, 1, 0]",0,"[코디, 전신]","[캐주얼, 스트릿]","{'전': 27, '니': 1}","{'야': 25, '실': 5}",0,0
4,@_mgi_closet_,"[2856667809394379675.jpg, 2858777754566839754....","[전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, 전, ...","[실, 실, 실, 실, 야, 야, 실, 실, 실, 실, 실, 실, 야, 실, 실, ...",NaN,"[0, 0, 0, 0]",0,"[전신, 야외, 코디]","[댄디, 스트릿]",{'전': 30},"{'실': 25, '야': 5}",0,0


In [31]:
#구도1
total = []
for i in range(len(tag)):
    encode = []
    for j in structure1:
        c= tag['촬영구도 카운트'][i][j]
        encode.append(c)
    total.append(encode)
    
tag['촬영구도 원핫']= total

In [32]:
total = []
for i in range(len(tag)):
    encode = []
    for j in structure2:
        c= tag['촬영장소 카운트'][i][j]
        encode.append(c)
    total.append(encode)
    
tag['촬영장소 원핫']= total

남녀별로 카테고리가 다르므로 각자 따로 one-hot encoding 한다.

In [34]:
#태그
for i in range(len(tag)):
    if tag['성별'][i] == 0:
        encode = []
        for j in cate_m:
            if j in tag['태그'][i]:
                encode.append(1)
            else:
                encode.append(0)
        tag['태그'][i] = encode
    else:
        encode = []
        for j in cate_w:
            if j in tag['태그'][i]:
                encode.append(1)
            else:
                encode.append(0)
        tag['태그'][i] = encode

C:\Users\minje\AppData\Local\Temp\ipykernel_24068\1549738105.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['태그'][i] = encode
C:\Users\minje\AppData\Local\Temp\ipykernel_24068\1549738105.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag['태그'][i] = encode


In [38]:
train = tag[['id','팔로워','성별','태그','촬영구도 원핫']] #촬영장소는 사용하지 않기로 함

one-hot encoding이 잘 되었다.

In [36]:
tag['성별'].value_counts()

1    56
0    45
Name: 성별, dtype: int64

# 3. 업체 인플루언서 추천 함수 정의

구도, 팔로워, 태그 등의 내적값 높으면 더 높은점수?

In [52]:
import numpy as np

def recommend_for_company(sex, follower, structure, category): #one-hot encoding된 리스트 input
    if int(sex) == 0:
        #성별 필터링
        filtered_influencers = train[train['성별']==0].reset_index(drop=True)
    else:
        filtered_influencers = train[train['성별']==1].reset_index(drop=True)
    #팔로워 필터링
    index=[]
    for i in range(len(filtered_influencers)):
        if np.dot(filtered_influencers['팔로워'][i],follower) != 0:
            index.append(i)
    filtered_influencers1 = filtered_influencers.iloc[index,:].reset_index(drop=True)
    #구도 필터링
    index1=[]
    for i in range(len(filtered_influencers1)):
        if np.dot(filtered_influencers1['촬영구도 원핫'][i],structure) != 0:
            index1.append(i)
    filtered_influencers2 = filtered_influencers1.iloc[index1,:].reset_index(drop=True)
    #태그 필터링
    index2=[]
    for i in range(len(filtered_influencers2)):
        if np.dot(filtered_influencers2['태그'][i],category) != 0:
            index2.append(i)
    filtered_influencers_final = filtered_influencers2.iloc[index2,:].reset_index(drop=True)

    #최종 점수 계산
    scores = []
    for i in range(len(filtered_influencers_final)):
        score = np.dot(filtered_influencers_final['팔로워'][i], follower) + np.dot(filtered_influencers_final['태그'][i], category) + np.dot(filtered_influencers_final['촬영구도 원핫'][i], structure)
        scores.append(score)
    filtered_influencers_final['final_score'] = scores
    filtered_influencers_final.sort_values('final_score', ascending=False, inplace=True)
    filtered_influencers_final.reset_index(drop=True, inplace=True)

    return filtered_influencers_final.iloc[:3,:]

test해보기

Input 값
1. sex = 남(0), 여(1)
2. follower = ['mega', 'macro', 'midtier', 'micro']
3. structure = ['니샷', '상반신', '셀카', '제품', '전신', '하반신']
4. category (남, 여에 따라 아래 두가지)
- cate_m(sex=0인 경우) = ['댄디', '스트릿', '스포츠', '아메리칸캐주얼', '캐주얼', '포멀']
- cate_w(sex=1인 경우) = ['걸리시', '레트로', '로맨틱', '스트릿', '스포츠', '시크', '캐주얼', '포멀']

In [53]:
# 팔로워(투자비, 광고비)는 중간정도이며 주로 전신사진이 많이 올라오는 인플루언서에게 협찬을 하고 싶으며,
# 주로 남성의류이며 스트릿하고 아메카지한 상품을 파는 업체에는 다음과 같은 인플루언서들을 추천한다.

recommend_for_company(0, [0,1,1,0], [0,0,0,0,1,0], [0,1,0,1,0,0])

,id,팔로워,성별,태그,촬영구도 원핫,final_score
0,@bejoon0,"[0, 0, 1, 0]",0,"[0, 1, 0, 1, 0, 0]","[0, 0, 0, 0, 30, 0]",33
1,@hodu__jwan,"[0, 0, 1, 0]",0,"[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 30, 0]",32
2,@j0ng_wo0,"[0, 0, 1, 0]",0,"[0, 1, 0, 1, 0, 0]","[0, 0, 0, 0, 28, 0]",31


이외에 추가적으로 이미지를 넣고 싶다면 소비자 인플루언서 추천모델을 추가로 실행하여 도출하면 된다.